In [ ]:
import os
import requests
import pandas as pd
from googleapiclient.discovery import build
import random
from collections import Counter
from tqdm import tqdm #for a visual progressbar in longer processes
from transformers import pipeline
import string
from innertube import InnerTube
import re
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


ModuleNotFoundError: No module named 'innertube'

In [ ]:
# Before displaying, make sure to mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install packages if needed
!pip install innertube
#pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00


# Main Functions | Final Code | Data Extraction

In [ ]:
# Collect random Video ID's | Initialize all required things

# Initialize the language detection pipeline once
model_ckpt = "papluca/xlm-roberta-base-language-detection"
language_detector = pipeline("text-classification", model=model_ckpt)

# Initialize the InnerTube client once
innertube_client = InnerTube(client_name='WEB')

def generate_random_video_id(mk):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=mk))

def search_youtube(query):
    PARAMS_TYPE_VIDEO = "EgIQAQ%3D%3D"
    return innertube_client.search(query=query, params=PARAMS_TYPE_VIDEO)

def filter_and_extract_videos(contents, unique_video_ids):
    video_infos = []
    for content in contents:
        video_info = content.get('videoRenderer', {})
        if not video_info:
            continue

        video_id = video_info.get('videoId', '')
        if video_id in unique_video_ids:
            continue  # Skip already processed videos

        publish_date = video_info.get('publishedTimeText', {}).get('simpleText', '')
        if "1 year ago" not in publish_date:
            continue  # Skip if the video was not published "1 year ago"

        title = video_info.get('title', {}).get('runs', [])[0].get('text', '')
        view_count_str = video_info.get('viewCountText', {}).get('simpleText', '').replace(' views', '').replace(',', '')

        try:
            view_count = int(view_count_str)
            if view_count <= 5000:
                continue
        except ValueError:
            continue  # Skip if view count is not determinable

        lang_predictions = language_detector(title, truncation=True)
        if lang_predictions[0]['label'] == "en":
            unique_video_ids.add(video_id)
            video_infos.append({
                'video_id': video_id,
                'title': title,
                'thumbnail_url': video_info.get('thumbnail', {}).get('thumbnails', [])[0].get('url', ''),
                'publish_date': publish_date,
                'view_count': view_count,
                'language': 'en'
            })

    return video_infos


In [ ]:
# Collect random Videos | Execute

desired_rows = 20000
characters_per_id = 4
video_data = []
unique_video_ids = set()

with tqdm(total=desired_rows) as pbar:
    while len(video_data) < desired_rows:
        vid_id = generate_random_video_id(characters_per_id)
        response = search_youtube(vid_id)
        contents = response.get('contents', {}).get('twoColumnSearchResultsRenderer', {}).get('primaryContents', {}).get('sectionListRenderer', {}).get('contents', [])[0].get('itemSectionRenderer', {}).get('contents', [])
        new_videos = filter_and_extract_videos(contents, unique_video_ids)
        video_data.extend(new_videos[:max(0, desired_rows - len(video_data))])

        # Update the progress bar only if new videos were added.
        if new_videos:
            pbar.update(len(new_videos))

video_df = pd.DataFrame(video_data)


# Store YT Video infro retrived with innertube library
video_df.to_csv('/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/04_Random_Dataset/yt_video_random.csv', index=False)

print(video_df.shape)
video_df

100%|██████████| 20000/20000 [7:01:35<00:00,  1.26s/it]


(20000, 6)


,video_id,title,thumbnail_url,publish_date,view_count,language
0,CaCc68x098Y,IS BUDGET NIGHTVISION ANY GOOD?? | WildGuarder...,https://i.ytimg.com/vi/CaCc68x098Y/hq720.jpg?s...,1 year ago,37134,en
1,woMFKz-NzSA,HOW THE DIRECTIVE ENDED FERRARI'S FIGHT FOR TH...,https://i.ytimg.com/vi/woMFKz-NzSA/hq720.jpg?s...,1 year ago,55287,en
2,_Qx0S_-J2fY,"One drawing, But 4 ✨Different✨ Styles! 👀 PART ...",https://i.ytimg.com/vi/_Qx0S_-J2fY/hq720_2.jpg...,1 year ago,46035752,en
3,EGSF1om4SAM,"Timi Zajc FALL at 161,5 m Willingen! Huge Jump...",https://i.ytimg.com/vi/EGSF1om4SAM/hq720.jpg?s...,1 year ago,273358,en
4,Nrk8sqZfsgI,It's pronounced GIF,https://i.ytimg.com/vi/Nrk8sqZfsgI/hq720.jpg?s...,1 year ago,11464212,en
...,...,...,...,...,...,...
19995,TgxOBzCo4ac,Must🤪Watch Amezing Special New Comedy Video 20...,https://i.ytimg.com/vi/TgxOBzCo4ac/hq720.jpg?s...,1 year ago,542803,en
19996,Uqsd6aMTK9Q,FFXIV Endwalker - Part 24 (Lv 87) - Thou Must ...,https://i.ytimg.com/vi/Uqsd6aMTK9Q/hq720.jpg?s...,1 year ago,22289,en
19997,gb8J0B-PNs8,How to Make Martha Stewart's Porchetta | Marth...,https://i.ytimg.com/vi/gb8J0B-PNs8/hq720.jpg?s...,1 year ago,54417,en
19998,Omm8pTF-4vU,ORAX - Crash (Full Album) 2023,https://i.ytimg.com/vi/Omm8pTF-4vU/hqdefault.j...,1 year ago,16314,en


In [ ]:
# Check unque video IDs from the random file result

# Path to the CSV file
csv_path = "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/04_Random_Dataset/yt_video_random.csv"

# Load the CSV file
df = pd.read_csv(csv_path)

# Count the number of unique video IDs
unique_video_ids_count = df['video_id'].nunique()

print(f"There are {unique_video_ids_count} unique video IDs in the dataset.")


There are 20000 unique video IDs in the dataset.


In [ ]:
# Split into 6 parts due to size of later job

import pandas as pd

# Path to the CSV file
csv_path = "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/04_Random_Dataset/yt_video_random.csv"

# Load the CSV file
df = pd.read_csv(csv_path)

# Calculate the size of each part
part_size = len(df) // 6  # Split into 6 parts

# Split the DataFrame into six parts
dfs = [
    df.iloc[i * part_size:(i + 1) * part_size] for i in range(5)  # First 5 parts
]
dfs.append(df.iloc[5 * part_size:])  # Last part includes the remainder

# Base path for new files
base_path = "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/04_Random_Dataset/00_Parts/"

# Save each part to a new CSV file with the specified path
for i, part_df in enumerate(dfs, start=1):
    new_file_path = f"{base_path}yt_video_random_part_{i}.csv"
    part_df.to_csv(new_file_path, index=False)


In [ ]:
# Get data via API using API Keys
# Use list of Video ID to get Information from offical Google YouTube API

csv_path = "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/04_Random_Dataset/00_Parts/yt_video_random_part_6.csv" #change parts from 1 - 6, since we might encounter the limitof YT API

df_initial_yt_search = pd.read_csv(csv_path)

# Initialize the YouTube API client
DEVELOPER_KEY = 'XXXX' #your key
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)


def parse_duration(duration):
    """Convert ISO 8601 duration format to seconds."""
    pattern = re.compile('P(?:(\d+)D)?T(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?')
    days, hours, minutes, seconds = pattern.match(duration).groups()
    days = int(days) if days else 0
    hours = int(hours) if hours else 0
    minutes = int(minutes) if minutes else 0
    seconds = int(seconds) if seconds else 0
    total_seconds = days * 86400 + hours * 3600 + minutes * 60 + seconds
    return round(total_seconds)


def get_category_names(regionCode='US'):
    """Fetch category names mapped by categoryId for a specified region."""
    category_names = {}
    request = youtube.videoCategories().list(
        part='snippet',
        regionCode=regionCode
    )
    response = request.execute()
    for item in response['items']:
        category_id = item['id']
        category_name = item['snippet']['title']
        category_names[category_id] = category_name
    return category_names

def get_video_and_channel_stats(video_ids):
    video_stats = []
    channel_ids = set()
    for video_id in video_ids:
        request = youtube.videos().list(
            part="snippet,statistics",
            id=video_id
        )
        response = request.execute()
        for item in response['items']:
            channel_id = item['snippet']['channelId']
            channel_ids.add(channel_id)
            duration = item['contentDetails']['duration']  # Extract duration
            duration_in_seconds = parse_duration(duration)  # Convert to seconds
            video_stats.append({
                'Video Title': item['snippet']['title'],
                'Video ID': video_id,
                'Channel ID': channel_id,
                'Video Description': item['snippet']['description'],
                'Video Duration': duration_in_seconds,  # Store duration
                'Video Views': item['statistics']['viewCount'],
                'Video Likes': item['statistics'].get('likeCount', 0),
                'Video Comments': item['statistics'].get('commentCount', 0),
                'Category ID': item['snippet']['categoryId']
            })
    channel_stats = get_channel_stats(list(channel_ids))
    return video_stats, channel_stats

def get_channel_stats(channel_ids):
    """Fetch statistics for a list of channel IDs."""
    channel_stats = {}
    for channel_id in channel_ids:
        request = youtube.channels().list(
            part="snippet,statistics",
            id=channel_id
        )
        response = request.execute()
        for item in response['items']:
            channel_stats[channel_id] = {
                'Channel Name': item['snippet']['title'],
                'Subscriber Count': item['statistics'].get('subscriberCount', 0)
            }
    return channel_stats

# Fetch category names
category_names = get_category_names(regionCode='US')

# Extract Video IDs from dataframe
video_ids = df_initial_yt_search['video_id'].tolist()

# Get stats for videos and their channels
video_stats, channel_stats = get_video_and_channel_stats(video_ids)

# Merge video stats with channel stats and normalize data
for video in video_stats:
    channel_id = video['Channel ID']
    video['Channel Name'] = channel_stats[channel_id]['Channel Name']
    subscriber_count = int(channel_stats[channel_id]['Subscriber Count'])
    video['Subscriber Count'] = subscriber_count
    video['Video Views Normalized'] = round(int(video['Video Views']) / subscriber_count,4) if subscriber_count else 0
    video['Video Likes Normalized'] = round(int(video['Video Likes']) / subscriber_count,4) if subscriber_count else 0
    video['Video Comments Normalized'] = round(int(video['Video Comments']) / subscriber_count,4) if subscriber_count else 0
    video['Category Name'] = category_names.get(video['Category ID'], 'Unknown')

# Filter out videos without a description ergo no SHORTS
video_stats_filtered = [video for video in video_stats if video['Video Description'].strip()]

# Create DataFrame
df_final = pd.DataFrame(video_stats)

print(f"DataFrame head:\n{df_final.head()}")
print(f"DataFrame dimensions: {df_final.shape}")


# !!! CAUTION CHANGE FOR EACH PART OF CSV !!!
# Store YT Video Info retrived with YouTube API
df_final.to_csv('/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_6.csv', index=False, escapechar='\\')


DataFrame head:
                                         Video Title     Video ID  \
0  #Wednesday meditation #sarahjakesroberts #Moti...  HYUt58-Gp9w   
1      (67) PR SAD - Lightwork Freestyle | Pressplay  kKFbEKGdCaQ   
2                 20th Century Boys | Complete Story  4h7H-iFq9sM   
3  1978 Winnebago Minnie Winnie 20RG Class C Moto...  U2dck4mpBOk   
4  The Goodwill Outlet Was Packed With Christmas ...  GlR5jK-R1M0   

                 Channel ID  \
0  UCBrykSPAsupifVbm8HKR4NQ   
1  UCwxK6Ba2IzJXw8gW2ZPjHSw   
2  UCEFqirZNiJ75sKhzVUPm0Cw   
3  UCkHs4xRYdEbVRfzgDYw7mbA   
4  UCCgQ6d80YMfRVN3mA3CxlNA   

                                   Video Description Video Views Video Likes  \
0  Wednesday meditation is focusing on the new da...        5487          13   
1  TikTok : shorturl.at/DFJPS\nSUBSCRIBE NOW: htt...     1193371       13979   
2  Thanks to ExpressVPN for sponsoring today's vi...       92036        2522   
3  https://www.bullyanrvs.com/product/used-1978-w...        82

In [ ]:
# Merge single parts back together

# Paths of the four parts to merge
parts_paths = [
    "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_1.csv",
    "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_2.csv",
    "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_3.csv",
    "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_4.csv",
    "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_5.csv",
    "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_6.csv"
]

# Load and concatenate the DataFrame from each part
dfs = [pd.read_csv(path) for path in parts_paths]
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_path = "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/yt_video_stats_final.csv"
merged_df.to_csv(merged_path, index=False)


# Main Functions | Final Code | Thumbnail Extraction

In [ ]:
# In hight (480x360) resolution which is the highest that is always available

# Load CSV with the video IDs (again from the six parts)
csv_path = "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/03_Final_Dataset/00_Parts/yt_video_stats_final_part_6.csv"

df = pd.read_csv(csv_path)

# Base folder path for saving thumbnails
base_folder_path = "/content/drive/MyDrive/08_Studies/01_HSLU/04_Semester_4/00_MasterThesis/01_Data/00_YT_thumbnails"


# Initialize the YouTube API client
DEVELOPER_KEY = 'XXX' #your key
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)


def download_thumbnail(url, folder_path, filename):
    """Download the thumbnail and save it to the specified folder with a specific filename."""
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    file_path = os.path.join(folder_path, f'{filename}.jpg')
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_path, 'wb') as file:
            file.write(response.content)

def fetch_and_save_thumbnails(video_ids, base_folder_path):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)

    # Wrap video_ids with tqdm for a progress bar
    for video_id in tqdm(video_ids, desc="Downloading Thumbnails"):
        request = youtube.videos().list(
            part='snippet',
            id=video_id
        )
        response = request.execute()
        for item in response['items']:
            thumbnail_url = item['snippet']['thumbnails'].get('high', {}).get('url', 'No Thumbnail')
            if thumbnail_url != 'No Thumbnail':
                # Save each thumbnail directly in the base folder, named after the video ID
                download_thumbnail(thumbnail_url, base_folder_path, video_id)

    print("Download complete!")

# Assuming 'df' is your DataFrame and it contains a column named 'Video ID'
video_ids = df['Video ID'].unique()

# Fetch and save the thumbnails
fetch_and_save_thumbnails(video_ids, base_folder_path)


Download complete!
